In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
folder_path = '/content/drive/MyDrive/Colab Notebooks/MBD_smt6'

In [3]:
# Import library yang dibutuhkan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import joblib

# Untuk preprocessing dan feature engineering
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Set random seed untuk reproducibility
np.random.seed(42)

In [4]:
import pandas as pd

# Lokasi file di Google Drive
file_path = '/content/drive/MyDrive/Colab Notebooks/MBD_smt6/abalone.csv'

# Load dataset
df = pd.read_csv(file_path)

# Cek data
df.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [5]:
# ===============================================================
# 1. EXPLORATORY DATA ANALYSIS (EDA)
# ===============================================================
# Overview Dataset
print("Dataset Overview:")
print(f"Shape: {df.shape}")
print(df.info())
print("\nDescriptive Statistics:")
print(df.describe())

# Cek missing values
print("\nMissing Values:")
print(df.isnull().sum())

Dataset Overview:
Shape: (4177, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Sex            4177 non-null   object 
 1   Length         4177 non-null   float64
 2   Diameter       4177 non-null   float64
 3   Height         4177 non-null   float64
 4   WholeWeight    4177 non-null   float64
 5   ShuckedWeight  4177 non-null   float64
 6   VisceraWeight  4177 non-null   float64
 7   ShellWeight    4177 non-null   float64
 8   Rings          4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB
None

Descriptive Statistics:
            Length     Diameter       Height  WholeWeight  ShuckedWeight  \
count  4177.000000  4177.000000  4177.000000  4177.000000    4177.000000   
mean      0.523992     0.407881     0.139516     0.828742       0.359367   
std       0.120093     0.099240     0.041827     0.49

In [6]:
# Visualisasi distribusi target (Rings)
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.histplot(df['Rings'], kde=True)
plt.title('Distribusi Rings')
plt.xlabel('Rings')

plt.subplot(1, 2, 2)
sns.histplot(np.log1p(df['Rings']), kde=True)
plt.title('Distribusi Log(Rings)')
plt.xlabel('Log(Rings)')

plt.tight_layout()
plt.savefig('rings_distribution.png')
plt.close()

In [7]:
# Cek normalitas distribusi target (Rings)
_, p_value = stats.shapiro(df['Rings'].sample(min(500, len(df))))
print(f"\nShapiro-Wilk Test p-value for Rings: {p_value}")
print(f"Is Rings normally distributed? {'Yes' if p_value > 0.05 else 'No'}")


Shapiro-Wilk Test p-value for Rings: 2.6946250574972578e-15
Is Rings normally distributed? No


In [8]:
# Korelasi antar fitur numerik
numeric_cols = ['Length', 'Diameter', 'Height', 'WholeWeight',
                'ShuckedWeight', 'VisceraWeight', 'ShellWeight', 'Rings']
plt.figure(figsize=(10, 8))
correlation_matrix = df[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.savefig('correlation_matrix.png')
plt.close()

In [9]:
# Visualisasi pengaruh jenis kelamin terhadap Rings
plt.figure(figsize=(6, 5))
sns.boxplot(x='Sex', y='Rings', data=df)
plt.title('Rings by Sex')
plt.savefig('rings_by_sex.png')
plt.close()

# Scatter plot antara fitur numerik dan Rings
plt.figure(figsize=(18, 10))

for i, col in enumerate(numeric_cols[:-1]):  # exclude 'Rings'
    plt.subplot(2, 4, i+1)
    sns.scatterplot(x=col, y='Rings', data=df)
    plt.title(f'{col} vs Rings')

plt.tight_layout()
plt.savefig('numerical_features_vs_rings.png')
plt.close()

In [10]:
# ===============================================================
# 2. FEATURE ENGINEERING (for abalone.csv)
# ===============================================================
import pandas as pd
import numpy as np

# Salin dataframe asli
df_engineered = df.copy()

# Simpan kolom Sex untuk keperluan target encoding (jangan hapus)
# Tambahkan One-Hot Encoding Manual (tanpa menghapus kolom asli)
df_engineered['Sex_I'] = (df_engineered['Sex'] == 'I').astype(int)
df_engineered['Sex_M'] = (df_engineered['Sex'] == 'M').astype(int)
# Catatan: Sex_F dianggap baseline (tidak perlu kolom)

# Fitur interaksi tambahan
df_engineered['Length_to_Diameter'] = df_engineered['Length'] / df_engineered['Diameter']
df_engineered['Weight_to_Length'] = df_engineered['WholeWeight'] / df_engineered['Length']
df_engineered['Shell_to_WholeWeight'] = df_engineered['ShellWeight'] / df_engineered['WholeWeight']
df_engineered['InnerWeight'] = (
    df_engineered['ShuckedWeight'] +
    df_engineered['VisceraWeight'] +
    df_engineered['ShellWeight']
)
df_engineered['Height_to_Length'] = df_engineered['Height'] / df_engineered['Length']


In [11]:
# ===============================================================
# 3. PREPROCESSING AND PREPARING DATA
# ===============================================================
# Tentukan fitur dan target
features = [
    'Length', 'Diameter', 'Height', 'WholeWeight', 'ShuckedWeight',
    'VisceraWeight', 'ShellWeight', 'Sex',  # kategorikal
    'Length_to_Diameter', 'Weight_to_Length',
    'Shell_to_WholeWeight', 'InnerWeight', 'Height_to_Length'
]
target = 'Rings'

# Pisahkan X dan y
X = df_engineered[features]
y = df_engineered[target]

# Split data latih dan uji
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Cek normalitas target (Rings)
_, p_value_rings = stats.shapiro(y_train.sample(min(500, len(y_train))))
print(f"\nShapiro-Wilk Test p-value for Rings (training set): {p_value_rings}")
use_standard_scaler = (p_value_rings > 0.05)
print(f"Using StandardScaler: {use_standard_scaler}")


Shapiro-Wilk Test p-value for Rings (training set): 4.899955142314748e-14
Using StandardScaler: False


In [12]:
# 5. Target Encoding untuk kolom kategorikal (Sex)
categorical_cols = ['Sex']
for col in categorical_cols:
    encoding_map = y_train.groupby(X_train[col]).mean().to_dict()
    X_train[f'{col}_encoded'] = X_train[col].map(encoding_map)
    X_test[f'{col}_encoded'] = X_test[col].map(encoding_map)

    # Handle kategori tidak dikenal di data uji
    if X_test[f'{col}_encoded'].isna().any():
        global_mean = y_train.mean()
        X_test[f'{col}_encoded'].fillna(global_mean, inplace=True)

In [13]:
# Definisikan preprocessing pipeline
numeric_features = [
    'Length', 'Diameter', 'Height', 'WholeWeight', 'ShuckedWeight',
    'VisceraWeight', 'ShellWeight', 'Length_to_Diameter',
    'Weight_to_Length', 'Shell_to_WholeWeight',
    'InnerWeight', 'Height_to_Length'
]
categorical_features = ['Sex']
encoded_features = [f'{col}_encoded' for col in categorical_features]

# Pilih scaler berdasarkan normalitas target
scaler = StandardScaler() if use_standard_scaler else MinMaxScaler()

# Pipeline untuk fitur numerik
numeric_transformer = Pipeline(steps=[
    ('scaler', scaler)
])

# Pipeline untuk fitur kategorikal
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first', sparse_output=False))
])

# Gabungkan dalam ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('encoded', 'passthrough', encoded_features)
    ])

# Buat pipeline preprocessing akhir
preprocess_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Fit dan transform data
preprocess_pipeline.fit(X_train)
X_train_preprocessed = preprocess_pipeline.transform(X_train)
X_test_preprocessed = preprocess_pipeline.transform(X_test)

# Log transformasi target (jika target tidak normal)
if not use_standard_scaler:
    print("Applying log transformation to Rings...")
    y_train_log = np.log1p(y_train)
    y_test_log = np.log1p(y_test)
else:
    y_train_log = y_train
    y_test_log = y_test

Applying log transformation to Rings...


In [14]:
# ===============================================================
# 4. MODEL TRAINING AND EVALUATION
# ===============================================================
# Create and train model
model = LinearRegression()
model.fit(X_train_preprocessed, y_train_log)

# Make predictions
y_train_pred_log = model.predict(X_train_preprocessed)
y_test_pred_log = model.predict(X_test_preprocessed)

# If log transformation was applied, reverse it for evaluation
if not use_standard_scaler:
    y_train_pred = np.expm1(y_train_pred_log)
    y_test_pred = np.expm1(y_test_pred_log)
else:
    y_train_pred = y_train_pred_log
    y_test_pred = y_test_pred_log

# Evaluate model
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("\nModel Evaluation:")
print(f"Training RMSE: {train_rmse:.2f}")
print(f"Test RMSE: {test_rmse:.2f}")
print(f"Training R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")

# Cross-validation for model stability
# We'll use a simple approach with the preprocessed data
cv_scores = cross_val_score(model, X_train_preprocessed, y_train_log,
                           cv=5, scoring='neg_root_mean_squared_error')
cv_rmse = -cv_scores.mean()
print(f"\nCross-Validation RMSE: {cv_rmse:.2f}")


Model Evaluation:
Training RMSE: 2.18
Test RMSE: 2.23
Training R²: 0.5368
Test R²: 0.5419

Cross-Validation RMSE: 0.18


In [15]:
# Plot actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_test_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual Charges')
plt.ylabel('Predicted Charges')
plt.title('Actual vs Predicted Charges')
plt.savefig('actual_vs_predicted.png')
plt.close()

In [16]:
# Plot residuals
residuals = y_test_pred - y_test
plt.figure(figsize=(10, 6))
plt.scatter(y_test_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Charges')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.savefig('residuals.png')
plt.close()

In [17]:
# Feature importance analysis
# Get feature names from preprocessing pipeline
num_features = numeric_features
cat_features = []
for name, _, cols in preprocessor.transformers_:
    if name == 'cat':
        cat_encoder = preprocessor.named_transformers_['cat'].named_steps['onehot']
        cat_features = list(cat_encoder.get_feature_names_out(categorical_features))
encoded_features = [f for f in encoded_features]

all_features = num_features + cat_features + encoded_features

# Create a DataFrame of coefficients
coef_df = pd.DataFrame({'Feature': all_features[:len(model.coef_)], 'Coefficient': model.coef_})
coef_df = coef_df.sort_values('Coefficient', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Coefficient', y='Feature', data=coef_df.head(15))
plt.title('Top 15 Important Features')
plt.tight_layout()
plt.savefig('important_features.png')
plt.close()

In [18]:
# ===============================================================
# 5. SAVE MODEL AND PREPROCESSING PIPELINE (for abalone.csv)
# ===============================================================
import joblib

# Kelas model prediksi abalone
class AbalonePredictionModel:
    def __init__(self, model, preprocessor, categorical_cols, encoding_maps, log_transform=False):
        self.model = model
        self.preprocessor = preprocessor
        self.categorical_cols = categorical_cols
        self.encoding_maps = encoding_maps
        self.log_transform = log_transform
        self.global_mean = None  # Untuk kategori tak dikenal saat prediksi

    def prepare_features(self, data):
        """
        Menyiapkan fitur termasuk fitur rekayasa untuk prediksi
        """
        if isinstance(data, dict):
            data = pd.DataFrame([data])

        df = data.copy()

        # Tambahkan fitur hasil rekayasa (manual)
        df['Length_to_Diameter'] = df['Length'] / df['Diameter']
        df['Weight_to_Length'] = df['WholeWeight'] / df['Length']
        df['Shell_to_WholeWeight'] = df['ShellWeight'] / df['WholeWeight']
        df['InnerWeight'] = (
            df['ShuckedWeight'] +
            df['VisceraWeight'] +
            df['ShellWeight']
        )
        df['Height_to_Length'] = df['Height'] / df['Length']

        # One-hot manual untuk 'Sex' (tidak hapus kolom asli)
        df['Sex_I'] = (df['Sex'] == 'I').astype(int)
        df['Sex_M'] = (df['Sex'] == 'M').astype(int)

        # Target encoding
        for col in self.categorical_cols:
            encoding_map = self.encoding_maps.get(col, {})
            df[f'{col}_encoded'] = df[col].map(encoding_map)

            if df[f'{col}_encoded'].isna().any():
                df[f'{col}_encoded'].fillna(self.global_mean, inplace=True)

        return df

    def predict(self, data):
        """
        Lakukan prediksi pada data baru
        """
        df_prepared = self.prepare_features(data)
        X_transformed = self.preprocessor.transform(df_prepared)
        pred_log = self.model.predict(X_transformed)

        if self.log_transform:
            pred = np.expm1(pred_log)
        else:
            pred = pred_log

        return pred

# ========================================
# Simpan komponen model
# ========================================
encoding_maps = {}
categorical_cols = ['Sex']  # hanya satu kolom kategorikal

# Buat encoding map dari data latih
for col in categorical_cols:
    encoding_maps[col] = y_train.groupby(X_train[col]).mean().to_dict()

# Buat instance model prediksi
abalone_model = AbalonePredictionModel(
    model=model,
    preprocessor=preprocess_pipeline,
    categorical_cols=categorical_cols,
    encoding_maps=encoding_maps,
    log_transform=(not use_standard_scaler)
)

# Set global mean target
abalone_model.global_mean = y_train.mean()

# Simpan model dan pipeline ke file
joblib.dump(abalone_model, 'abalone_prediction_model.joblib')
print("\nModel saved successfully!")



Model saved successfully!


In [19]:
# ===============================================================
# 6. DISPLAY LINEAR REGRESSION FORMULA (for abalone.csv)
# ===============================================================
import matplotlib.pyplot as plt

# Ambil koefisien dan intercept dari model
coefficients = model.coef_
intercept = model.intercept_

# Ambil nama fitur dari pipeline
feature_names = []
for name, transformer, cols in preprocessor.transformers_:
    if name == 'num':
        feature_names.extend(cols)
    elif name == 'cat':
        # Cek apakah onehot digunakan
        cat_encoder = transformer.named_steps['onehot']
        encoded_names = cat_encoder.get_feature_names_out(cols)
        feature_names.extend(encoded_names)
    elif name == 'encoded':
        feature_names.extend(cols)

# Pastikan jumlah nama fitur sama dengan jumlah koefisien
feature_names = feature_names[:len(coefficients)]

# Urutkan fitur berdasarkan nilai absolut koefisien (besar pengaruh)
coef_feature_pairs = sorted(zip(coefficients, feature_names), key=lambda x: abs(x[0]), reverse=True)

# Cek apakah target dilog-transform
target_formula = "log(Rings)" if not use_standard_scaler else "Rings"

# Buat formula lengkap
formula = f"{target_formula} = {intercept:.4f}"
for coef, feature in coef_feature_pairs:
    formula += f" {'+' if coef >= 0 else '-'} {abs(coef):.4f} × {feature}"

print("\nLinear Regression Formula:")
print(formula)

# Buat versi ringkas (top 10 fitur)
top_n = 10
top_features = coef_feature_pairs[:top_n]
simple_formula = f"{target_formula} = {intercept:.4f}"
for coef, feature in top_features:
    simple_formula += f" {'+' if coef >= 0 else '-'} {abs(coef):.4f} × {feature}"
simple_formula += " + ..."

print("\nSimplified Formula (Top 10 features):")
print(simple_formula)

# Simpan formula ke file
with open('linear_regression_formula.txt', 'w') as f:
    f.write("Complete Linear Regression Formula:\n")
    f.write(formula + "\n\n")
    f.write("Simplified Formula (Top 10 features):\n")
    f.write(simple_formula + "\n\n")
    f.write("Feature Importance (Absolute Coefficient Value):\n")
    for i, (coef, feature) in enumerate(coef_feature_pairs, 1):
        f.write(f"{i}. {feature}: {abs(coef):.6f}\n")

print("\nFormula saved to 'linear_regression_formula.txt'")

# ===============================================================
# Visualisasi Koefisien (Top 15 Fitur Penting)
# ===============================================================
plt.figure(figsize=(12, 8))

# Ambil top 15 koefisien numerik
top_plot_features = [(f, c) for c, f in coef_feature_pairs[:15] if isinstance(c, (int, float))]
features, coefs = zip(*top_plot_features)
colors = ['red' if c < 0 else 'blue' for c in coefs]

# Buat bar plot horizontal
plt.barh(range(len(features)), [abs(c) for c in coefs], color=colors)
plt.yticks(range(len(features)), features)
plt.xlabel('Absolute Coefficient Value')
plt.title('Top 15 Important Features in Linear Regression Model')
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Tambahan penjelasan warna
plt.text(max([abs(c) for c in coefs]) * 0.7, -1.5,
         'Red = Negative Impact, Blue = Positive Impact',
         fontsize=10, ha='center')

plt.tight_layout()
plt.savefig('regression_coefficients.png')
plt.close()



Linear Regression Formula:
log(Rings) = 1.4503 + 1.9555 × Length + 1.7904 × Weight_to_Length - 1.7622 × ShuckedWeight + 1.2811 × Height_to_Length - 0.9893 × Diameter - 0.9005 × Length_to_Diameter - 0.7533 × InnerWeight + 0.7469 × ShellWeight + 0.7130 × Shell_to_WholeWeight - 0.4935 × Height - 0.1981 × VisceraWeight + 0.1730 × WholeWeight + 0.0152 × Sex_encoded + 0.0143 × Sex_M - 0.0068 × Sex_I

Simplified Formula (Top 10 features):
log(Rings) = 1.4503 + 1.9555 × Length + 1.7904 × Weight_to_Length - 1.7622 × ShuckedWeight + 1.2811 × Height_to_Length - 0.9893 × Diameter - 0.9005 × Length_to_Diameter - 0.7533 × InnerWeight + 0.7469 × ShellWeight + 0.7130 × Shell_to_WholeWeight - 0.4935 × Height + ...

Formula saved to 'linear_regression_formula.txt'


In [20]:
# ===============================================================
# 7. ANALISIS HASIL (Abalone Dataset)
# ===============================================================
print("\n===== ANALISIS HASIL =====")

print("\n1. Analisis Distribusi Data")
if not use_standard_scaler:
    print("- Variabel target 'Rings' tidak berdistribusi normal (p-value < 0.05)")
    print("- Menggunakan transformasi logaritma pada variabel target")
    print("- Menggunakan MinMaxScaler untuk feature scaling")
else:
    print("- Variabel target 'Rings' berdistribusi normal (p-value > 0.05)")
    print("- Tidak perlu transformasi logaritma")
    print("- Menggunakan StandardScaler untuk feature scaling")

print("\n2. Analisis Performa Model")
print(f"- RMSE pada data test: {test_rmse:.2f}")
print(f"- R² pada data test: {test_r2:.4f}")
print(f"- Nilai R² {test_r2:.4f} menunjukkan bahwa model dapat menjelaskan {test_r2*100:.2f}% variasi pada jumlah cincin ('Rings') abalone")

if test_r2 >= 0.75:
    print("- Model memiliki kemampuan prediksi yang baik (R² > 0.75)")
elif test_r2 >= 0.5:
    print("- Model memiliki kemampuan prediksi yang cukup (R² > 0.5)")
else:
    print("- Model memiliki kemampuan prediksi yang kurang baik (R² < 0.5)")

print("\n3. Analisis Faktor-faktor yang Mempengaruhi Rings")
print("- Berdasarkan koefisien model, fitur-fitur yang paling berpengaruh adalah:")
for i in range(min(5, len(coef_df))):
    print(f"  {i+1}. {coef_df.iloc[i]['Feature']}: {coef_df.iloc[i]['Coefficient']:.4f}")



===== ANALISIS HASIL =====

1. Analisis Distribusi Data
- Variabel target 'Rings' tidak berdistribusi normal (p-value < 0.05)
- Menggunakan transformasi logaritma pada variabel target
- Menggunakan MinMaxScaler untuk feature scaling

2. Analisis Performa Model
- RMSE pada data test: 2.23
- R² pada data test: 0.5419
- Nilai R² 0.5419 menunjukkan bahwa model dapat menjelaskan 54.19% variasi pada jumlah cincin ('Rings') abalone
- Model memiliki kemampuan prediksi yang cukup (R² > 0.5)

3. Analisis Faktor-faktor yang Mempengaruhi Rings
- Berdasarkan koefisien model, fitur-fitur yang paling berpengaruh adalah:
  1. Length: 1.9555
  2. Weight_to_Length: 1.7904
  3. Height_to_Length: 1.2811
  4. ShellWeight: 0.7469
  5. Shell_to_WholeWeight: 0.7130
